In [5]:
import pandas as pd
import string
from nltk.corpus import stopwords

In [6]:
df_spamcollection=pd.read_csv('C:\\Users\\shash\\OneDrive\\Desktop\\SpamCollection',sep='\t',names=['response','message'])

In [7]:
df_spamcollection.head()

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
df_spamcollection.describe()

,response,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [9]:
df_spamcollection.groupby( 'response' ).describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [10]:
df_spamcollection['length']=df_spamcollection['message'].apply(len)

In [11]:
df_spamcollection.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [12]:
import nltk
nltk.download('stopwords')
def message_text_process(mess):
    no_punctuation=[char for char in mess if char not in string.punctuation]
    no_punctuation=''.join(no_punctuation)
    return [word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
df_spamcollection['message'].head(5).apply(message_text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
bagofwords=CountVectorizer(analyzer=message_text_process).fit(df_spamcollection['message'])

In [16]:
print(len(bagofwords.vocabulary_))

11425


In [17]:
messagebag=bagofwords.transform(df_spamcollection['message'])

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer().fit(messagebag)

In [19]:
tfidf=tfidf_transformer.transform(messagebag)
print(tfidf.shape)

(5572, 11425)


In [20]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model=MultinomialNB().fit(tfidf,df_spamcollection['response'])

In [22]:
message=df_spamcollection['message'][4]
bagofwordsmessage=bagofwords.transform([message])
tfidf_=tfidf_transformer.transform(bagofwordsmessage)
print('predicted',spam_detect_model.predict(tfidf_)[0])
print('expected',df_spamcollection.response[4])

predicted ham
expected ham
